In [1]:
import math
from analysis_tools import *
import bokeh
from bokeh.plotting import *
output_notebook()
import matplotlib.pyplot as plt

Loading BokehJS ...

In [2]:
runs = runsFromFolder('data_and_control')

In [3]:
test = runs[3]
plot = figure(title="Run")
plot.line([t.time for t in test.data], [t.motorTemp for t in test.data])
show(plot)

In [4]:
findSpikes(test)

[]

In [5]:
keys = list(filter(lambda x: not '_' in x, dir(test.data[0])))

In [6]:
eqTime = 60

def equilibriumSection(run, cutoffIndex):
    i = cutoffIndex
    cutoffTime = run.data[i].time
    while i > 0 and (cutoffTime - eqTime) < run.data[i].time:
        i -= 1

    return run.data[i:cutoffIndex]

In [7]:
eqData = equilibriumSection(test, findSpikes(test)[0])

# TODO: get and plot equilibrium section for all spikes

IndexError: list index out of range

In [ ]:
for key in keys:
    p = figure(title=key)
    p.line([t.time for t in eqData], [getattr(t, key) for t in eqData])
    show(p)

In [ ]:
#tmpD = test.data[i-100:i]
#for key in keys:
#    p = figure(title=key)
#    p.line([t.time for t in tmpD], [getattr(t, key) for t in tmpD])
#    show(p)

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot([t.motorTemp for t in eqData])

In [ ]:
from scipy import stats

In [ ]:
times = [t.time for t in eqData]
temps = [t.motorTemp for t in eqData]
stats.linregress(times, temps)

In [ ]:
import numpy

In [ ]:
numpy.mean([t.motorTemp for t in eqData])

In [ ]:
numpy.std([t.motorTemp for t in eqData])

In [ ]:
eqTests = list(filter(lambda x: 'EqTest' in x.file, runs))
[x.file for x in eqTests]

In [ ]:

n = len(eqData)
n_2 = math.floor(n / 2)

mBar = 0
for i in range(0, n_2):
    x_i = eqData[i]
    x_ni = eqData[n_2 + i]
    m_i = (x_ni.motorTemp - x_i.motorTemp) / (x_ni.time - x_i.time)
    mBar += m_i
    
abs(mBar)

In [ ]:
def highlightSegment(run, hlStartTime, hlEndTime):
    hlStartIndex = -1
    hlEndIndex = -1

    i = 0
    while hlStartIndex < 0:
        if (test.data[i].time >= hlStartTime):
            hlStartIndex = i
        i += 1
    while hlEndIndex < 0:
        if (test.data[i].time >= hlEndTime):
            hlEndIndex = i
        i += 1

    plot = figure()

    plot.line([t.time for t in run.data], [t.motorTemp for t in run.data])
    hlData = run.data[hlStartIndex:hlEndIndex]
    plot.scatter([t.time for t in hlData], [t.motorTemp for t in hlData], line_color='orange')

    show(plot)

In [ ]:
hlTimes = (eqData[0].time, eqData[-1].time)

highlightSegment(test, hlTimes[0], hlTimes[1])

In [ ]:
class SlopeCutoff:
    def __init__(self):
        self.eqSize = 60 * 2
        self.sampleQueue = [(0.0, 0.0) for x in range(0, self.eqSize-1)]
        self.mQueue = [0 for x in range(0, math.floor((self.eqSize/2) - 1))]
        self.mSum = 0
        self.seen = 0
        self.mThresh = 1/self.eqSize

        self.record_mSum = list()

    def cutoffWindow(self, currentSample):
        self.seen += 1
    
        self.sampleQueue.pop(0)
        self.sampleQueue.append( (currentSample.time, currentSample.motorTemp) )
        self.mSum -= self.mQueue.pop(0)
                
        halfPt = self.sampleQueue[int((self.eqSize/2)-1)]
        yDelta = currentSample.motorTemp - halfPt[1]
        xDelta = currentSample.time - halfPt[0]
        m = yDelta / xDelta
        self.mQueue.append(m)
        self.mSum += m
    
        self.record_mSum.append( (currentSample.time, currentSample.motorTemp, self.mSum, m) )
    
        # If the queue is shorter than 60 seconds, keep running the motor.
        if self.seen < self.eqSize:
            return False
        else:
            return (abs(self.mSum) < self.mThresh) and (max([x[1] for x in self.sampleQueue]) - min([x[1] for x in self.sampleQueue])) < 1

In [ ]:
#i = 0
#sc = SlopeCutoff()
#while i < len(test.data) and not sc.cutoffWindow(test.data[i]):
#    i += 1
#    
#i

In [ ]:
#plot1 = figure()
#plot1.line([x.time for x in test.data], [x.motorTemp for x in test.data])
#show(plot1)

#plot2 = figure(x_range=plot1.x_range)
#plot2.line([x[0] for x in sc.record_mSum], [x[1] for x in sc.record_mSum])
#show(plot2)

In [ ]:
#highlightSegment(test, sc.sampleQueue[0][0], sc.sampleQueue[-1:][0][0])

In [ ]:
runsWithEqSpike = filter(lambda r: len(r.data) > 120 and max(r.data, key=lambda o: o.pwm).pwm > 1400, eqTests)

In [ ]:
def findNewEqInRun(run):
    i = 0
    sc = SlopeCutoff()
    while i < len(run.data) and not sc.cutoffWindow(run.data[i]):
        i += 1

    highlightSegment(run, sc.sampleQueue[0][0], sc.sampleQueue[-1:][0][0])
    
    #plot = figure()
    #plot.scatter([x[0] for x in sc.record_mSum], [x[1] for x in sc.record_mSum])
    #show(plot)
    
    return sc

In [ ]:
errs = list()
evaluation = list()
for run in runsWithEqSpike:
    print()
    print(run.file)
    try:
        sc = findNewEqInRun(run)
        temps = [x[1] for x in sc.sampleQueue]
        print('min temp: ' + str(min(temps)))
        print('max temp: ' + str(max(temps)))
        print('temp range: ' + str(max(temps) - min(temps)))
        print('samples: ' + str(len(sc.sampleQueue)))
        
        m_actual = stats.linregress([x[0] for x in sc.sampleQueue], [x[1] for x in sc.sampleQueue])
        
        evaluation.append(sc, m_actual)
        
    except:
        errs.append(run.file)

In [ ]:
import seaborn as sns

# Eq Proof Files

In [ ]:
proofRuns = runsFromFolder('eq_proof')

In [ ]:
errs = list()
evaluation = list()
for run in proofRuns:
    print()
    print(run.file)
    try:
        sc = findNewEqInRun(run)
        temps = [x[1] for x in sc.sampleQueue]
        print('min temp: ' + str(min(temps)))
        print('max temp: ' + str(max(temps)))
        print('temp range: ' + str(max(temps) - min(temps)))
        print('samples: ' + str(len(sc.sampleQueue)))
        
        m_actual = stats.linregress([x[0] for x in sc.sampleQueue], [x[1] for x in sc.sampleQueue])
        
        evaluation.append(sc, m_actual)
        
    except:
        errs.append(run.file)

## Toy Example of Slope Approximation

In [ ]:
xtoy = [0,6,12,18,24,30,36,42,48,54]
ytoy = [79.6, 79.3,80.3, 79.7, 79.2, 79.5, 79.7, 79.9, 79.5, 79.6]

In [ ]:
toyAvg = sum(ytoy) / len(ytoy)

In [ ]:
toyReg = stats.linregress(xtoy, ytoy)

In [ ]:
plot = figure(title='Time vs Temperature', x_axis_label='Time (s)', y_axis_label='Temperature (°F)', x_range=[-3,60])
plot.scatter(xtoy, ytoy, size=7)
plot.line(xtoy, ytoy, line_width=3)
show(plot)

In [ ]:
cs = ['red', 'orange', 'green', 'blue', 'purple', 'grey']
pts = list(zip(xtoy, ytoy))

def line(p1, p2):
    return [p1, p2], (p2[1] - p1[1]) / (p2[0] - p1[0])

t = [ line(pts[0], pts[4]),
line(pts[1], pts[5]),
line(pts[2], pts[6]),
line(pts[3], pts[7]),
line(pts[4], pts[8]),
line(pts[5], pts[9]) ]

mbar = 0
for i in range(0, len(t)):
    l,m = t[i]
    plot.line([x[0] for x in l], [x[1] for x in l], line_color=cs[i], line_dash='dotted', line_width=2)
    mbar += m

show(plot)

In [ ]:
toyReg.slope, mbar

In [ ]:
toyPlt = [(x, toyAvg+((x-30)*mbar)) for x in range(0, 60)]
xToyPlt = [x[0] for x in toyPlt]
yToyPlt = [x[1] for x in toyPlt]
plot.line(xToyPlt, yToyPlt, line_dash='dashed', line_color='red', line_width=3)
show(plot)

In [ ]:
toyRegPlt = [(x, toyReg.intercept+(x*toyReg.slope)) for x in range(0, 60)]
xToyReg = [x[0] for x in toyRegPlt]
yToyReg = [x[1] for x in toyRegPlt]
plot.line(xToyReg, yToyReg, line_dash='dashed', line_color='black', line_width=5)
show(plot)

In [ ]:
max(yToyPlt) - min(yToyPlt)

In [ ]:
mTest = 0
for i in range(0, len(xtoy)):
    for j in range(i+1, len(xtoy)):
        mTest += (ytoy[j] - ytoy[i]) / (xtoy[j] - ytoy[i])

mTest

In [ ]:
print('\n'.join([str(x[1]) for x in t]))

In [ ]:
sum([x[1] for x in t])